 Сегодня поработаем с романом "Sense and Sensibility" Джейн Остен.

In [ ]:
with open ('SenseAndSensibilityJAu.txt', encoding='utf-8') as txt: #положите файл для обработки в корневую папку для вашего удобства
    text = txt.read()
    print(len(text.split())) #наивная токенизация

Довольно большой роман, но на самом деле там много всякой ненужной нам информации, поэтому сделаем чуть больше предобработки, чем обычно. Проверим:

In [ ]:
print(text[:1550]) #cрез по первым 1550 символам

In [ ]:
print(text[-19000:]) #19 тысяч символов с конца

Давайте удалим эти вставки в начале и конце. 

In [ ]:
import re

In [ ]:
text_no_info = re.sub(r'\n', ' ', text)
text_no_info = re.sub('^The Project Gutenberg eBook of Sense and Sensibility.*?CHAPTER L', '', text_no_info)
print(text_no_info[:1550])

In [ ]:
text_no_info = re.sub(r'\*\*\* END OF THE PROJECT GUTENBERG EBOOK SENSE AND SENSIBILITY \*\*\*\n.*', '', text_no_info)
print(text_no_info[-19000:])

In [ ]:
text_no_info = re.sub(' +', ' ', text_no_info) #косметически уберем лишние пробельчики
text_no_info

Лемматизировать английский текст через pymorphy (как мы ранее делали с русскоязычным текстом) бесполезно. Можно, например, через nltk, stanza или другие инструменты.

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()

In [ ]:
lemmatized_words = [lemmatizer.lemmatize(word) for word in word_tokenize(text_no_info[:1000]) if word not in stopwords_en]
print(lemmatized_words)

Как вы видите, результат похож скорее на стемминг. Проверим stanza.

In [ ]:
import stanza
stanza.download("en")
nlp_stanza = stanza.Pipeline(lang="en", processors="tokenize, pos, lemma")

In [ ]:
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

In [ ]:
text_stanza = nlp_stanza(text_no_info[:1000])
print(text_stanza) #результат

In [ ]:
full_text_lemmas = []
for sentence in text_stanza.iter_tokens():
  #  print(sentence)
    for word in sentence.words:
        #print(word)
        #print(word.lemma.lower())
        word_lem = word.lemma.lower()
        if word_lem not in stopwords_en:
            full_text_lemmas.append(word_lem)
       #     print('yay!')
      #  else:
         #   print ('nay')
full_text_lemmas

Это было скорее напоминание о stanza. Это довольно медленный инструмент и мы не можем пользоваться им на занятии, но для исследований, требующих высокой точности и при этом позволяющих оставить компьютер размечать некоторое время - это одно из лучших решений. Теперь вернемся к WordNetLemmatizer.

In [ ]:
from tqdm import tqdm

In [ ]:
lem_text_no_info = [lemmatizer.lemmatize(word) for word in tqdm(word_tokenize(text_no_info.lower())) if word not in stopwords_en]
print(len(lem_text_no_info))

In [ ]:
lem_text_no_info[:100]

Нарисуем dispersion plot с помощью nltk, чтобы посмотреть распределения слов по графику (этот кусочек позаимствован у Д. Скоринкина). 

In [ ]:
from nltk import Text as nltk_text

In [ ]:
text_no_info_nltk = nltk_text(lem_text_no_info)
type(text_no_info_nltk)

In [ ]:
lemmatized_nltk_format.dispersion_plot(['joy', 'sorrow', 'death', 'life', 'dead', 'happy'])

Также мы можем самостоятельно разбить текст на квартили, децили или любые другие блоки (чанки) и посмотреть, насколько распределены те или иные единицы по корпусу.

In [ ]:
text_join = ' '.join(lem_text_no_info)
length = len(text_join)
chunk_size = length // 10 #количество блоков
remainder = length % 10 #количество блоков
chunks = []
for i in range(9): #в скобочках нужно указать такое число, на которое вы хотите разделить текст, -1. Здесь мы делим на децили
    chunks.append(text_join[i * chunk_size : (i + 1) * chunk_size])
    chunks.append(text_join[9 * chunk_size:]) 


print(len(chunks))

In [ ]:
print(round(len(lem_text_no_info) / 10)) #длина чанков

In [ ]:
chunks = []
chunk_size = round(len(lem_text_no_info) / 10)  # Целочисленное деление для размера чанка
chunk_beg = 0

for i in range(10):
    chunk_switch = chunk_beg + chunk_size
    if i == 9:  # Последний чанк включает остаток
        chunk_switch = len(lem_text_no_info)
    chunk = lem_text_no_info[chunk_beg:chunk_switch]
    chunks.append(' '.join(chunk))
    chunk_beg = chunk_switch

In [ ]:
print(len(chunks[7]))
print(len(chunks[1]))
print(len(chunks[9]))

Проверим, что последний чанк действительно последний:

In [ ]:
chunks[9]

А вот теперь можно посчитать разную статистику по нашим децилям. Например, найдем количество слов 'sorrow' в каждом из чанков:

In [ ]:
print(len(chunks))

In [ ]:
our_lady_of_sorrows = []
for chunk in chunks:
    counter = 0
    chunk_tok = chunk.split()
    for word in chunk_tok:
        if word == 'sorrow':
            counter += 1
    our_lady_of_sorrows.append(counter)
our_lady_of_sorrows    

Поместим это на график:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.plot(our_lady_of_sorrows)
plt.xlabel("Дециль книги")
plt.ylabel("Количество единиц слова")
plt.title("Sorrow")
plt.xticks(range(len(our_lady_of_sorrows))) #чтобы показывались все децили
plt.show()

Этого нам мало. Давайте извлечен несколько слов и разобъем по децилям. Обернем наш код выше в функцию:

In [ ]:
def word_chunk_counter(data, word_s):
    word_in_chunks = []
    for chunk in data:
        counter = 0
        chunk_tok = chunk.split()
        for word in chunk_tok:
            if word == str(word_s):
                counter += 1
        word_in_chunks.append(counter)
    return word_in_chunks

Проверим для начала на 'sorrow':

In [ ]:
our_lady_of_sorrows_2 = word_chunk_counter(chunks, 'sorrow')
print(our_lady_of_sorrows == our_lady_of_sorrows_2)

Отлично, всё работает правильно! Найдем ещё 5 слов.

In [ ]:
dolor = word_chunk_counter(chunks, 'sorrow')
gaudium = word_chunk_counter(chunks, 'joy')
mors = word_chunk_counter(chunks, 'death')
nativitas = word_chunk_counter(chunks, 'birth')
felix = word_chunk_counter(chunks, 'happy')
tristus = word_chunk_counter(chunks, 'sad')

In [ ]:
print(dolor)
print(gaudium)
print(mors)
print(nativitas)
print(felix)
print(tristus)

Нанесем их все на один график.

In [ ]:
all_words = [dolor, gaudium, mors, nativitas, felix, tristus]
all_words

In [ ]:
x = range(len(all_words[0])) # ось Х одинакова для всех линий
line_names = ['sorrow', 'joy', 'death', 'birth', 'happy', 'sad']
plt.figure(figsize=(10, 6))
sns.set_theme(style="ticks")


for i, line_data in enumerate(all_words):
    sns.lineplot(x=x, y=line_data, label=line_names[i], marker='o')

plt.xlabel("Дециль книги", fontsize=12)
plt.ylabel("Количество единиц слова", fontsize=12)
plt.title("Распределение частот шести заданных слов", fontsize=14)
plt.legend() # показываем легенду
plt.xticks(range(len(our_lady_of_sorrows)))
plt.show()

Также иногда делают один график, но со множеством разных ячеек-графиков, получается что-то вроде плитки шоколада.

In [ ]:
x = range(len(all_words[0]))
line_names = ['sorrow', 'joy', 'death', 'birth', 'happy', 'sad']
fig, axes = plt.subplots(2, 3, figsize=(15, 8)) 
axes = axes.flatten()

sns.set_theme(style="ticks")
colors = sns.color_palette("plasma", len(all_words))

for i, line_data in enumerate(all_words):
    ax = axes[i]
    sns.lineplot(x=x, y=line_data, marker='o', ax=ax, color=colors[i]) 
    ax.set_title(line_names[i]) 
    ax.set_xlabel("Дециль книги")
    ax.set_ylabel("Количество единиц слова")
    ax.set_xticks(range(len(our_lady_of_sorrows)))

plt.tight_layout() 
plt.suptitle("Распределение исследуемых слов", fontsize=16)
plt.subplots_adjust(top=0.92)
plt.show()

**Задание 1.** Поэксперементируйте с другими словами. Какой результат у вас получается?

**Задание 2.** Переделайте исследование так, чтобы считались не абсолютные значения, а относительные.

Вернемся в сладостные объятия русского языка. Поисследуем Библию.

In [ ]:
with open('Torah.txt', encoding='utf-8') as txt:
    text_torah = txt.read()
    text_torah = re.sub(r'\n', ' ', text_torah[:1000])
    text_torah = re.sub(r'[^а-яА-Я -]', '', text_torah)
    text_torah = re.sub(r' +', ' ', text_torah)
    print(text_torah)

In [ ]:
with open('Torah.txt', encoding='utf-8') as txt:
    text_torah = txt.read()
    text_torah = re.sub(r'\n', ' ', text_torah)
    text_torah = re.sub(r'[^а-яА-Я -]', '', text_torah)
    text_torah = re.sub(r' +', ' ', text_torah)

with open('Joshua.txt', encoding='utf-8') as txt:
    text_joshua = txt.read()
    text_joshua = re.sub(r'\n', ' ', text_joshua)
    text_joshua = re.sub(r'[^а-яА-Я -]', '', text_joshua)
    text_joshua = re.sub(r' +', ' ', text_joshua)

with open('New_testament.txt', encoding='utf-8') as txt:
    text_nt = txt.read()
    text_nt = re.sub(r'\n', ' ', text_nt)
    text_nt = re.sub(r'[^а-яА-Я -]', '', text_nt)
    text_nt = re.sub(r' +', ' ', text_nt)

In [ ]:
print(len(text_torah.split()))
print(len(text_joshua.split()))
print(len(text_nt.split()))

На децили разбивать не будем, просто сравним три книги по употреблению лексем и граммем.

In [ ]:
from pymorphy3 import MorphAnalyzer #загрузим анализатор
morph = MorphAnalyzer() #сохраним его в переменную, чтобы заработал

In [ ]:
tagged_joshua = []
for word in word_tokenize(text_joshua):
    parsed_word = morph.parse(word)[0]
    if parsed_word.tag.POS == 'VERB':
        tagged_joshua.append((parsed_word.normal_form, parsed_word.tag.tense))

Можно переделать это в функцию:

In [ ]:
def tagger_text(data, tag):
    tagged_list = []
    for word in word_tokenize(data):
        parsed_word = morph.parse(word)[0]
        if parsed_word.tag.POS == str(tag):
            tagged_list.append((parsed_word.normal_form, parsed_word.tag.tense))
    return tagged_list

In [ ]:
tagged_joshua_2 = tagger_text(text_joshua, 'VERB')

In [ ]:
tagged_joshua == tagged_joshua_2

Отлично. Затегируем две оставшиеся книги:

In [ ]:
tagged_torah = tagger_text(text_torah, 'VERB')
tagged_nt = tagger_text(text_nt, 'VERB')

In [ ]:
tagged_nt[:10]

In [ ]:
import collections

In [ ]:
verb_lemmas_torah = [item[0] for item in tagged_torah]
count = collections.Counter(verb_lemmas_torah)
count

In [ ]:
rel_frequency_list = []
for item, freq in tqdm(count.items()):
    rel_freq = freq / len(tagged_torah) * 100
    rel_frequency_list.append((item, freq, rel_freq))
rel_frequency_list.sort(key=lambda x: x[1], reverse=True)
rel_frequency_list

Обернем в функцию:

In [ ]:
def rel_freq_list(data):
    verb_lemmas = [item[0] for item in data]
    count = collections.Counter(verb_lemmas)
    rel_frequency_list = []
    for item, freq in tqdm(count.items()):
        rel_freq = freq / len(tagged_torah) * 100
        rel_frequency_list.append((item, freq, rel_freq))
    rel_frequency_list.sort(key=lambda x: x[1], reverse=True)
    return rel_frequency_list

In [ ]:
rel_freq_list(tagged_torah) == rel_frequency_list

Посмотрим на списки двух других частей:

In [ ]:
rel_freq_list(tagged_joshua)

In [ ]:
rel_freq_list(tagged_nt)

Можно визуализировать:

In [ ]:
items = [item for item, freq, rel_freq in rel_freq_list(tagged_nt)[:30]]
frequencies = [rel_freq for item, freq, rel_freq in rel_freq_list(tagged_nt)[:30]]

plt.figure(figsize=(10, 6))
plt.bar(items, frequencies)
plt.xlabel("Глагол")
plt.ylabel("Частота глагола")
plt.title("Частотное распределение глаголов")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

Посчитаем относительное (относительного глаголов, а не всех токенов) количество слов "убить".

In [ ]:
all_books = [tagged_torah, tagged_joshua, tagged_nt]
counters = []
for book in tqdm(all_books):
    num_verbs = len(book)
    counter = 0
    for word in book:
        if word[0] == 'убить':
            counter += 1
    counters.append(counter / num_verbs * 100)

In [ ]:
counters

И снова обернем в функцию:

In [ ]:
def bible_token_rel_counter(data, token):
    counters_tok = []
    for book in tqdm(all_books):
        num_verbs = len(book)
        counter = 0
        for word in book:
            if word[0] == str(token):
                counter += 1
        counters_tok.append(counter / num_verbs * 100)
    return (counters_tok)

In [ ]:
bible_token_rel_counter(all_books, 'убить') == counters

Отлично! Получим подсчёты слов "родить" и "миловать".

In [ ]:
kill = bible_token_rel_counter(all_books, 'убить')
born = bible_token_rel_counter(all_books, 'родить')
spare = bible_token_rel_counter(all_books, 'прощать')

In [ ]:
born

In [ ]:
spare

Теперь нанесем все три слова на график.

In [ ]:
words = [kill, born, spare]
words

In [ ]:
x = range(len(words[0]))
line_names = ['убить', 'родить', 'прощать']
book_names = ['Тора', 'Книга Иисуса Навина', 'Новый Завет']
plt.figure(figsize=(10, 6))
sns.set_theme(style="ticks")


for i, line_data in enumerate(words):
    sns.lineplot(x=x, y=line_data, label=line_names[i], marker='o')

plt.xlabel("Книга библии", fontsize=12)
plt.ylabel("Относительная частота", fontsize=12)
plt.title("Распределение частот слов", fontsize=14)
plt.legend() # показываем легенду
plt.xticks(x, book_names)
plt.show()

И на трех разных графиках:

In [ ]:
x = range(len(words[0]))
line_names = ['убить', 'родить', 'прощать']
book_names = ['Тора', 'Книга Иисуса Навина', 'Новый Завет']
fig, axes = plt.subplots(1, 3, figsize=(15,5)) 
axes = axes.flatten()

sns.set_theme(style="ticks")
colors = sns.color_palette("plasma", len(words))

for i, line_data in enumerate(words):
    ax = axes[i]
    sns.lineplot(x=x, y=line_data, marker='o', ax=ax, color=colors[i]) 
    ax.set_title(line_names[i]) 
    ax.set_xlabel("Книга библии")
    ax.set_ylabel("Относительная частота")
    ax.set_xticks(x, book_names)

plt.tight_layout() 
plt.suptitle("Распределение частот слов", fontsize=16)
plt.subplots_adjust(top=0.9)
plt.show()

**Задание 3.** Измените код выше так, чтобы он извлекал существительные. Исследуйте несколько значимых (на ваш взгляд) существительных. 

Изучим граммемы. Давайте нанесем на график глаголы прошедшего времени в каждой из частей.

In [ ]:
def bible_gram_rel_counter(data, grammeme):
    counters_gramm = []
    for book in tqdm(data):
        num_verbs = len(book)
        counter = 0
        for word in book:
            if word[1] == str(grammeme):
                counter += 1
            else: 
                continue
        counters_gramm.append(counter / num_verbs * 100)
    return (counters_gramm)

In [ ]:
past_verbs = bible_gram_rel_counter(all_books, 'past')

Осталось нанести на график.

In [ ]:
x = range(len(past_verbs))
book_names = ['Тора', 'Книга Иисуса Навина', 'Новый Завет']
plt.figure(figsize=(8, 6))
sns.lineplot(x=x, y=past_verbs, marker='o', label='Прошедшее время')
plt.xlabel("Часть Библии")
plt.ylabel("Доля глагола")
plt.title("Частота глаголов прошедшего времени")
plt.xticks(x, book_names)
plt.legend()
plt.show()

**Задание 4 (для самых отчаянных).** Переделайте код так, что он оставил только существительные и их падежи. Сделайте два графика:
1. На первом только один плот с несколькими линями, где каждая линия - конкретный падеж;
2. Сделайте плитку из плотов, где на каждом плоте одна линия конкретного падежа.